In [21]:
# Decision tree classifier - Grid method
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [23]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [24]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [25]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [26]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [27]:
#split into training set and test
# In regression, data splitting is not required. Whereas in Classification it is required
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [28]:
# If the predicted values is between the defined range, then standard sccalar is required
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [29]:
# Import the DecisionTree classifier model
from sklearn.tree import DecisionTreeClassifier

In [30]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [31]:
# Import the GridsearchCV model
from sklearn.model_selection import GridSearchCV

# configuring the parameters specific to this model
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['int','float','sqrt','log2'],
              'splitter':['best','random']} 


# Scoring is important in this step. This is equal to R_Square in the regression
# This parameter is suitable for both imbalanced and balanced data
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
# In place of f1_weighted, other parameters listed in the above page can be replaced and tested
grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search. Model gets created in this step
grid.fit(X_train, y_train)  



Fitting 5 folds for each of 16 candidates, totalling 80 fits


C:\Users\rhenaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
40 fits failed out of a total of 80.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\rhenaj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\rhenaj\AppData\Local\Packages\PythonSoftwareFound

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['int', 'float', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [32]:
# print best parameter after tuning 

re=grid.cv_results_

grid_predictions = grid.predict(X_test) 
   
# Importing the confusion matrix object
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)


# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [33]:
# Importing the f1_score
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'best'}: 0.9104477611940298


In [34]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [ 6 43]]


In [35]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.93      0.93      0.93        85
           1       0.88      0.88      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [36]:
# Importing the Receiver operating characteristic - Area under curve ( roc_auc ) score object 
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


np.float64(0.9034813925570228)

In [37]:
table=pd.DataFrame.from_dict(re)

In [38]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000954,0.000011,0.000000,0.000000,gini,int,best,"{'criterion': 'gini', 'max_features': 'int', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.000985,0.000143,0.000000,0.000000,gini,int,random,"{'criterion': 'gini', 'max_features': 'int', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.000730,0.000076,0.000000,0.000000,gini,float,best,"{'criterion': 'gini', 'max_features': 'float',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
3,0.000751,0.000151,0.000000,0.000000,gini,float,random,"{'criterion': 'gini', 'max_features': 'float',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
4,0.001973,0.000058,0.003202,0.000105,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.849794,0.866968,0.813179,0.906166,0.867097,0.860641,0.030082,1
5,0.001833,0.000295,0.003642,0.001225,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.887346,0.808392,0.759244,0.813179,0.793565,0.812345,0.041985,7
6,0.001768,0.000199,0.003162,0.000343,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.826263,0.850809,0.814409,0.813179,0.826499,0.826232,0.013522,5
7,0.001655,0.000122,0.002938,0.000191,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.851852,0.886792,0.793565,0.777291,0.808775,0.823655,0.040144,6
8,0.000633,0.000055,0.000000,0.000000,entropy,int,best,"{'criterion': 'entropy', 'max_features': 'int'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
9,0.000651,0.000045,0.000000,0.000000,entropy,int,random,"{'criterion': 'entropy', 'max_features': 'int'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9


In [ ]:
age_input=float(input("Age:"))
salary_input=float(input("BMI:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

In [ ]:
#Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])# change the paramter,play with it.
Future_Prediction=grid.predict([[age_input,salary_input,sex_male_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))